In [23]:
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
# from chatterbot.trainers import ListTrainer

chatbot = ChatBot("myBot")
chatbot.set_trainer(ChatterBotCorpusTrainer)

# 使用语料库训练它
# chatbot.train("chatterbot.corpus.english")
# chatbot.train("chatterbot.corpus.chinese")
chatbot.train("chatterbot.corpus.custom")
# train using list
# chatbot.set_trainer(ListTrainer)
# chatbot.train(movie_lines)

chineseMovie.yml Training: [####################] 100%
myown.yml Training: [####################] 100%


## Import necessary libraries

In [24]:
import io, os, subprocess, sys, re, codecs, time, datetime, requests, itchat
from itchat.content import *
from googleapiclient.discovery import build

In [25]:
# Here I read in my own API_KEY from a file, which is not shared in Github repository:
# with io.open('../../API_KEY.txt') as fp: 
#     for line in fp: APIKEY = line

# You need to un-comment below line and replace 'APIKEY' variable with your own GCP API key:
APIKEY='XXX'

In [26]:
# Below is for Google Language Tranlation API
service = build('translate', 'v2', developerKey=APIKEY)

# Below is for Google Natual Language Processing API
# nlp_service = build('language', 'v1', developerKey=APIKEY)
nlp_service = build('language', 'v1beta2', developerKey=APIKEY)

In [27]:
# parm_runtime_env_GCP = True
parm_runtime_env_GCP = False

Define image pre-processing functions

In [28]:
# Import the base64 encoding library.
import base64
# Pass the image data to an encoding function.
def encode_image(image_file):
    with io.open(image_file, "rb") as image_file:
        image_content = image_file.read()
# Python 2
    if sys.version_info[0] < 3:
        return base64.b64encode(image_content)
# Python 3
    else:
        return base64.b64encode(image_content).decode('utf-8')

Define control parameters for API

In [29]:
# API control parameter for Image API:
parm_image_maxResults = 10 # max objects or faces to be extracted from image analysis

# API control parameter for Language Translation API:
parm_translation_origin_language = 'zh' # original language in text: to be overwriten by TEXT_DETECTION
parm_translation_target_language = 'zh' # target language for translation: Chinese

# API control parameter for 自然语言处理：语义和情感分析
parm_nlp_extractDocumentSentiment = True # 情感分析 (Sentiment analysis)
parm_nlp_extractEntities = True          # 消息文字中名称实体的识别 (Name-Entity detection)
parm_nlp_extractEntitySentiment = False  # Only available in v1beta2. But Chinese language zh is not supported yet.
parm_nlp_extractSyntax = True            # 语句的语法分析 (Syntax / Grammar analysis)

## Recognize objects in image

[1] 物体名 (General Object)

In [30]:
# Running Vision API
# 'LABEL_DETECTION'
def ChatbotVS_LABEL_DETECTION(image_base64, API_type, maxResults):
    vservice = build('vision', 'v1', developerKey=APIKEY)
    request = vservice.images().annotate(body={
        'requests': [{
                'image': {
#                     'source': {
#                         'gcs_image_uri': IMAGE
#                     }
                      "content": image_base64
                },
                'features': [{
                    'type': API_type,
                    'maxResults': maxResults,
                }]
            }],
        })
    responses = request.execute(num_retries=3)
    
    image_analysis_reply = u'\nDid you send me\n你给我发的是'
    # 'LABEL_DETECTION'
    if responses['responses'][0] != {}:
        image_analysis_reply += responses['responses'][0]['labelAnnotations'][0]['description'] + u'？' \
            + '\n( Possibility 我有' +  str(round(responses['responses'][0]['labelAnnotations'][0]['score'], 3) * 100) + u'%的把握 )\n'
    else:
        image_analysis_reply += u'[ Nill 无结果 ]\n'
        
    return image_analysis_reply

[2] 地标名 (Landmark Object)

In [31]:
# Running Vision API
# 'LANDMARK_DETECTION'
def ChatbotVS_LANDMARK_DETECTION(image_base64, API_type, maxResults):
    vservice = build('vision', 'v1', developerKey=APIKEY)
    request = vservice.images().annotate(body={
        'requests': [{
                'image': {
#                     'source': {
#                         'gcs_image_uri': IMAGE
#                     }
                      "content": image_base64
                },
                'features': [{
                    'type': API_type,
                    'maxResults': maxResults,
                }]
            }],
        })
    responses = request.execute(num_retries=3)
    
    image_analysis_reply = u'\nThis place is\n这个地方貌似是'
    # 'LANDMARK_DETECTION'
    if responses['responses'][0] != {}:
        image_analysis_reply += responses['responses'][0]['landmarkAnnotations'][0]['description'] \
            + '\n( Possibility 可能性' +  str(round(responses['responses'][0]['landmarkAnnotations'][0]['score'], 3) * 100) + '% )\n'
    else:
        image_analysis_reply += u'[ Nill 无结果 ]\n'
        
    return image_analysis_reply

[3] 商标名 (Logo Object)

In [32]:
# Running Vision API
# 'LOGO_DETECTION'
def ChatbotVS_LOGO_DETECTION(image_base64, API_type, maxResults):
    vservice = build('vision', 'v1', developerKey=APIKEY)
    request = vservice.images().annotate(body={
        'requests': [{
                'image': {
#                     'source': {
#                         'gcs_image_uri': IMAGE
#                     }
                      "content": image_base64
                },
                'features': [{
                    'type': API_type,
                    'maxResults': maxResults,
                }]
            }],
        })
    responses = request.execute(num_retries=3)
    
    image_analysis_reply = u'\nThis mark is\n这个商标好像是'
    # 'LOGO_DETECTION'
    if responses['responses'][0] != {}:
        image_analysis_reply += responses['responses'][0]['logoAnnotations'][0]['description'] \
            + '\n( Possibility 可能性' +  str(round(responses['responses'][0]['logoAnnotations'][0]['score'], 3) * 100) + '% )\n'
    else:
        image_analysis_reply += u'[ Nill 无结果 ]\n'
        
    return image_analysis_reply

## OCR: Extract text from image

In [33]:
# Running Vision API
# 'TEXT_DETECTION'
def ChatbotVS_TEXT_DETECTION(image_base64, API_type, maxResults):
    vservice = build('vision', 'v1', developerKey=APIKEY)
    request = vservice.images().annotate(body={
        'requests': [{
                'image': {
#                     'source': {
#                         'gcs_image_uri': IMAGE
#                     }
                      "content": image_base64
                },
                'features': [{
                    'type': API_type,
                    'maxResults': maxResults,
                }]
            }],
        })
    responses = request.execute(num_retries=3)
    image_analysis_reply = u'\nI found text\n我找到文字: '
    # 'TEXT_DETECTION'
    if responses['responses'][0] != {}:
        image_analysis_reply += responses['responses'][0]['textAnnotations'][0]['description']
        ##############################################################################################################
        #                                        translation of detected text                                        #
        ##############################################################################################################
        parm_translation_origin_language = responses['responses'][0]['textAnnotations'][0]['locale']
        # Call translation if parm_translation_origin_language is not parm_translation_target_language
        if parm_translation_origin_language != parm_translation_target_language:
            inputs=[responses['responses'][0]['textAnnotations'][0]['description']] # TEXT_DETECTION OCR results only
            outputs = service.translations().list(source=parm_translation_origin_language, 
                                                  target=parm_translation_target_language, q=inputs).execute()
            image_analysis_reply += outputs['translations'][0]['translatedText'] + '\n'
        ##############################################################################################################
    else:
        image_analysis_reply += u'[ Nill 无结果 ]\n'
        
    return image_analysis_reply

## Recognize human face

In [34]:
# Running Vision API
# 'FACE_DETECTION'
def ChatbotVS_FACE_DETECTION(image_base64, API_type, maxResults):
    vservice = build('vision', 'v1', developerKey=APIKEY)
    request = vservice.images().annotate(body={
        'requests': [{
                'image': {
#                     'source': {
#                         'gcs_image_uri': IMAGE
#                     }
                     "content": image_base64
#                     'content': str(image_base64.decode("utf-8"))
                },
                'features': [{
                    'type': API_type,
                    'maxResults': maxResults,
                }]
            }],
        })
    responses = request.execute(num_retries=3)
    image_analysis_reply = u'\nI found your emotions\n你们的表情被我发现了:\n'
    # 'FACE_DETECTION'
    if responses['responses'][0] != {}:
        for i in range(len(responses['responses'][0]['faceAnnotations'])):
            image_analysis_reply += u'\nFace No.' + str(i+1) + u'第' + str(i+1) + u'张脸:\n'
            
            joy_detect = responses['responses'][0]['faceAnnotations'][i][u'joyLikelihood']
            if joy_detect != 'VERY_UNLIKELY':
                image_analysis_reply += u'Joy 喜悦, ' + u'probability 可能性 :' + joy_detect + '\n'
                
            anger_detect = responses['responses'][0]['faceAnnotations'][i][u'angerLikelihood']
            if anger_detect != 'VERY_UNLIKELY':
                image_analysis_reply += u'Anger 愤怒, ' + u'probability 可能性 :' + anger_detect + '\n'
            
            sorrow_detect = responses['responses'][0]['faceAnnotations'][i][u'sorrowLikelihood']
            if sorrow_detect != 'VERY_UNLIKELY':
                image_analysis_reply += u'Sorrow 悲伤, ' + u'probability 可能性 :' + sorrow_detect + '\n'
                
            surprise_detect = responses['responses'][0]['faceAnnotations'][i][u'surpriseLikelihood']
            if surprise_detect != 'VERY_UNLIKELY':
                image_analysis_reply += u'Surprise 惊奇, ' + u'probability 可能性 :' + surprise_detect + '\n'
                
            headwear_detect = responses['responses'][0]['faceAnnotations'][i][u'headwearLikelihood']
            if headwear_detect != 'VERY_UNLIKELY':
                image_analysis_reply += u'Headwear 头饰, ' + u'probability 可能性 :' + headwear_detect + '\n'
                
            blurred_detect = responses['responses'][0]['faceAnnotations'][i][u'blurredLikelihood']
            if blurred_detect != 'VERY_UNLIKELY':
                image_analysis_reply += u'Blurred 模糊, ' + u'probability 可能性 :' + blurred_detect + '\n'
                
            underExposed_detect = responses['responses'][0]['faceAnnotations'][i][u'underExposedLikelihood']
            if underExposed_detect != 'VERY_UNLIKELY':
                image_analysis_reply += u'UnderExposed 欠曝光, ' + u'probability 可能性 :' + underExposed_detect + '\n'
    else:
        image_analysis_reply += ''
            
    return image_analysis_reply

## Explicit Content Detection

In [35]:
# Running Vision API
# 'SAFE_SEARCH_DETECTION'
def ChatbotVS_SAFE_SEARCH_DETECTION(image_base64, API_type, maxResults):
    vservice = build('vision', 'v1', developerKey=APIKEY)
    request = vservice.images().annotate(body={
        'requests': [{
                'image': {
#                     'source': {
#                         'gcs_image_uri': IMAGE
#                     }
                     "content": image_base64
#                     'content': str(image_base64.decode("utf-8"))
                },
                'features': [{
                    'type': API_type,
                    'maxResults': maxResults,
                }]
            }],
        })
    responses = request.execute(num_retries=3)
    image_analysis_reply = u'\nI also detected some bad content\n我还检测到不良内容:\n'
    # 'SAFE_SEARCH_DETECTION'
    if responses['responses'][0] != {}:
        
        Adult_detect = responses['responses'][0]['safeSearchAnnotation'][u'adult']
        if Adult_detect != ('VERY_UNLIKELY' or 'UNLIKELY'):
            image_analysis_reply += u'Adult 成人, ' + u'probability 可能性 :' + Adult_detect + '\n'
            
        Violence_detect = responses['responses'][0]['safeSearchAnnotation'][u'violence']
        if Violence_detect != ('VERY_UNLIKELY' or 'UNLIKELY'):
            image_analysis_reply += u'Violence 暴力, ' + u'probability 可能性 :' + Violence_detect + '\n'
            
        Spoof_detect = responses['responses'][0]['safeSearchAnnotation'][u'spoof']
        if Spoof_detect != ('VERY_UNLIKELY' or 'UNLIKELY'):
            image_analysis_reply += u'Spoof 欺骗, ' + u'probability 可能性 :' + Spoof_detect + '\n'
            
        Medical_detect = responses['responses'][0]['safeSearchAnnotation'][u'medical']
        if Medical_detect != ('VERY_UNLIKELY' or 'UNLIKELY'):
            image_analysis_reply += u'Medical 医疗, ' + u'probability 可能性 :' + Medical_detect + '\n'           
    else:
        image_analysis_reply += ''
    return image_analysis_reply

## Text based language translation

In [36]:
def ChatbotVS_TEXT_TRANSLATION(text, origin_language_code, target_language_code):
    # Call translation if parm_translation_origin_language is not parm_translation_target_language
    if origin_language_code != target_language_code:
        outputs = service.translations().list(source=origin_language_code, 
                                              target=target_language_code, q=text).execute()
        outputs_zh = service.translations().list(source=origin_language_code, 
                                              target='zh', q=text).execute()
        
        translated_text = u' ---- I translated your text ----\n ---- 亲，我帮你翻译了一下 ---- \n\n'
        translated_text += u'( English Text 英文译文: )\n'
        translated_text += outputs['translations'][0]['translatedText'] + '\n'# + '----- End Translation -----\n'
        translated_text += u'( Chinese Text 中文译文: )\n'
        translated_text += outputs_zh['translations'][0]['translatedText'] + '\n'
    else:
        translated_text = text
        
    return translated_text

## NLP interface

In [37]:
# Running Speech API
def ChatbotVS_nlp(text, extractDocumentSentiment, extractEntities, extractEntitySentiment, extractSyntax): 
        
    request = nlp_service.documents().annotateText(body={
                "document":{
                    "type": "PLAIN_TEXT",
                    "content": text
                    },
                "features": {
                    "extractDocumentSentiment": extractDocumentSentiment,
                    "extractEntities": extractEntities,
                    "extractEntitySentiment": extractEntitySentiment, # only available in v1beta2
                    "extractSyntax": extractSyntax,
                    },
                "encodingType":"UTF8"
                })
    responses = request.execute(num_retries=3)        

    return responses

In [38]:
def ChatbotVS_nlp_generate_reply(responses):
    nlp_reply = u' ---- I analyzed your text ----\n ---- 嗯，我认真考虑了你的话 ----'
    
    # 1. 整篇消息文字的情感分析 (Sentiment analysis, Document level)
    nlp_reply += '\n'
    nlp_reply += '\n' + u'[  Whole text 全文 Lang 语种 : {} ]\n( 褒贬程度 : {} | 语彩累积 : {} )'.format(
            responses['language']
            , responses['documentSentiment']['score']
            , responses['documentSentiment']['magnitude']
        )

    # 2. 消息文字中语句的情感分析 (Sentiment analysis, Sentence level)           
    nlp_reply += '\n'
    for i in range(len(responses['sentences'])):
        nlp_reply += '\n' + u'[ Phrase 语句 {} : {} ]\n( 褒贬程度 : {} | 语彩累积 : {} )'.format(
              i+1
            , responses['sentences'][i]['text']['content']
            , responses['sentences'][i]['sentiment']['score']
            , responses['sentences'][i]['sentiment']['magnitude']
        )
                
    # 3. 消息文字中名称实体的识别 (Name-Entity detection)
    nlp_reply += '\n'
    for i in range(len(responses['entities'])): 
        nlp_reply += '\n' + u'[ Entity 实体 {} : {} ]\n  Category 类别 : {}\n  Importance 重要程度 : {}'.format(
              i+1
            , responses['entities'][i]['name']
            , responses['entities'][i]['type']
            , responses['entities'][i]['salience']
        )
        if 'sentiment' in responses['entities'][i]:
            nlp_reply += '\n' + u'  褒贬程度 : {}\n  语彩累积 : {}'.format(
                  responses['entities'][i]['sentiment']['score']
                , responses['entities'][i]['sentiment']['magnitude']
            )
        if responses['entities'][i]['metadata'] != {}:
            if 'wikipedia_url' in responses['entities'][i]['metadata']:
                nlp_reply += '\n  ' + responses['entities'][i]['metadata']['wikipedia_url']
                           
    # 4. 语句的语法分析 (Syntax / Grammar analysis)
#     nlp_reply += '\n'
#     for i in range(len(responses['tokens'])): 
#         nlp_reply += '\n' + str(responses['tokens'][i])
    
    return nlp_reply

## Login

In [39]:
itchat.auto_login(hotReload=True) # hotReload=True: 退出程序后暂存登陆状态。即使程序关闭，一定时间内重新开启也可以不用重新扫码。

In [40]:
# # 如果收到[TEXT, MAP, CARD, NOTE, SHARING]类的信息，会自动回复：
@itchat.msg_register([MAP, CARD, SHARING, RECORDING, ATTACHMENT, VIDEO]) # 位置、名片、分享、语音、文件、视频
def text_reply(msg):
    itchat.send(u'You sent me something strange, I will never open it!\n你给我发的都是些啥啊，我保证我不会打开的!', msg['FromUserName'])

# #如果收到新朋友的请求，会自动通过验证添加加好友，并主动打个招呼：幸会幸会！Nice to meet you!
@itchat.msg_register(FRIENDS)
def add_friend(msg):
    itchat.add_friend(**msg['Text']) # 该操作会自动将新好友的消息录入，不需要重载通讯录
    itchat.send_msg(u'Welcome to Chatbot of Vicious!\n\
    I can chat with you using movie subtitles.\n\
    Add a slash / to the beginning and I will translate it.\n\
    Add a star to the beginning and I will analyze it.\n\
    Send me a photo and I will identify it.\n\
    Try to send me some text (English/Chinese) or a photo.\n\
    欢迎来到Vicious的聊天机器人!\n\
    我可以回复电影台词跟你对话\n\
    句子开头加一个斜杠/我会帮你翻译\n\
    句子开头加一个星号*我会帮你分析\n\
    发给我一张图片我可以帮你识别\n\
    给我一段文字(中/英文)或者一张图片试试吧！', msg['RecommendInfo']['UserName'])

In [41]:
# @itchat.msg_register([PICTURE], isGroupChat=True)
@itchat.msg_register([PICTURE])
def download_files(msg):
    parm_translation_origin_language = 'zh' # will be overwriten by TEXT_DETECTION
    msg.download(msg.fileName)
#     print('\nDownloaded image file name is: %s' % msg['FileName'])
    image_base64 = encode_image(msg['FileName'])
    
    ##############################################################################################################
    #                                          call image analysis APIs                                          #
    ##############################################################################################################
    
    image_analysis_reply = u'---- Your image is interesting ----\n---- 哟，你的图片好有意思 ----\n'

    # 1. LABEL_DETECTION:
    LABEL_DETECTION = ChatbotVS_LABEL_DETECTION(image_base64, 'LABEL_DETECTION', parm_image_maxResults)
    if LABEL_DETECTION != (u'\nDid you send me\n你给我发的是' + u'[ Nill 无结果 ]\n'):
        image_analysis_reply += LABEL_DETECTION
    # 2. LANDMARK_DETECTION:
    LANDMARK_DETECTION = ChatbotVS_LANDMARK_DETECTION(image_base64, 'LANDMARK_DETECTION', parm_image_maxResults)
    if LANDMARK_DETECTION != (u'\nThis place is\n这个地方貌似是' + u'[ Nill 无结果 ]\n'):
        image_analysis_reply += LANDMARK_DETECTION
    # 3. LOGO_DETECTION:
    LOGO_DETECTION = ChatbotVS_LOGO_DETECTION(image_base64, 'LOGO_DETECTION', parm_image_maxResults)
    if LOGO_DETECTION != (u'\nThis mark is\n这个商标好像是' + u'[ Nill 无结果 ]\n'):
        image_analysis_reply += LOGO_DETECTION
    # 4. TEXT_DETECTION:
    TEXT_DETECTION = ChatbotVS_TEXT_DETECTION(image_base64, 'TEXT_DETECTION', parm_image_maxResults)
    if TEXT_DETECTION != (u'\nI found text\n我找到文字: ' + u'[ Nill 无结果 ]\n'):
        image_analysis_reply += TEXT_DETECTION
    # 5. FACE_DETECTION:
    FACE_DETECTION = ChatbotVS_FACE_DETECTION(image_base64, 'FACE_DETECTION', parm_image_maxResults)
    if FACE_DETECTION != (u'\nI found your emotions\n你们的表情被我发现了:\n'):
        image_analysis_reply += FACE_DETECTION
    # 6. SAFE_SEARCH_DETECTION:
    SAFE_SEARCH_DETECTION = ChatbotVS_SAFE_SEARCH_DETECTION(image_base64, 'SAFE_SEARCH_DETECTION', parm_image_maxResults)
    if SAFE_SEARCH_DETECTION != (u'\nI also detected some bad content\n我还检测到不良内容:\n'):
        image_analysis_reply += SAFE_SEARCH_DETECTION
    
    return image_analysis_reply

In [ ]:
# 如果收到[TEXT]类的信息，会自动回复：
@itchat.msg_register([TEXT]) # 文字
def text_reply(msg):
    # Translate recognised text to another language:
    parm_translation_origin_language = ''
    parm_translation_target_language = 'en'
    translated_text_reply = ''
    nlp_reply = ''
    res_reply = ''
    
    if msg['Text'][0] == ('/'):
        temp_translated_text_reply = msg['Text'][1:]
        translated_text_reply = ChatbotVS_TEXT_TRANSLATION(temp_translated_text_reply, 
                                                       parm_translation_origin_language, parm_translation_target_language)
    ##############################################################################################################
    # call NLP API:
    elif msg['Text'][0] == ('*'):
        text4nlp = msg['Content'][1:]
        nlp_responses = ChatbotVS_nlp(text4nlp
                            , parm_nlp_extractDocumentSentiment
                            , parm_nlp_extractEntities
                            , parm_nlp_extractEntitySentiment
                            , parm_nlp_extractSyntax)
        # Format NLP results:
        nlp_reply = ChatbotVS_nlp_generate_reply(nlp_responses)
    ##############################################################################################################
    # chat using corpus:
    else :
        res_reply = str(chatbot.get_response(msg['Content']))

    return translated_text_reply + nlp_reply + res_reply

In [ ]:
itchat.run()

Start auto replying.


In [ ]:
# interupt kernel, then logout
itchat.logout() # 安全退出